In [13]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geographiclib.geodesic import Geodesic
from shapely import LineString

In [19]:
def geodesic_line(olat, olon, dlat, dlon, return_distance=False):
    p=Geodesic.WGS84.Inverse(olat, olon, dlat, dlon)
    l=Geodesic.WGS84.Line(p['lat1'],p['lon1'],p['azi1'])

    if (p['s12'] >= 1000000):
        num = int (p['s12']/100000)  #number of waypoints depending on length
    else:
        num=10
    output=[]
    for i in range(num+1):
        b=l.Position(i*p['s12']/num, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
        output.append([b['lon2'],b['lat2'],b['azi2']])
    
    if return_distance:
        return LineString(output), p['s12']
    else:
        return LineString(output)

def get_openflights_data():
    airports = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',header=None)

    airports.columns = [
        'AirportID',
        'AirportName',
        'City',
        'Country',
        'IATA',
        'ICAO',
        'Latitude',
        'Longitude',
        'Altitude',
        'Timezone',
        'ZonaHoraria',
        'TZ',
        'Type',
        'Source'
    ]
    
    airports['geometry'] = gpd.points_from_xy(airports['Longitude'],airports['Latitude'],crs=4326)
    airports = gpd.GeoDataFrame(airports)

    airlines = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat',header=None)
    airlines.columns = [
        'AirlineID',
        'AirlineName',
        'Alias',
        'IATA',
        'ICAO',
        'Callsign',
        'Country',
        'Active'
    ]

    routes = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat',header=None)
    routes.columns = [
        'AirlineIATA(2)/ICAO(3)',
        'AirlineID',
        'SourceAirport',
        'SourceAirportID',
        'DestinationAirport',
        'DestinationAirportID',
        'Codeshare',
        'Stops',
        'Equipment'
    ]
    
    routes = routes[(routes['SourceAirportID']!='\\N')&(routes['DestinationAirportID']!='\\N')]
    routes['SourceAirportID'] = routes['SourceAirportID'].astype(int)
    routes['DestinationAirportID'] = routes['DestinationAirportID'].astype(int)

    planes = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat',header=None)
    planes.columns = [
        'Name',
        'IATA',
        'ICAO'
    ]

    countries = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/countries.dat',header=None)
    countries.columns = [
        'Name',
        'ISO_code',
        'DAFIF_code'
    ]
    
    airportcols = ['AirportID','Latitude','Longitude','City','Country']

    routes = routes.merge(
        airports[airportcols].rename(
            columns={col:f'Source{col}' for col in airportcols}
        ), 
    ).merge(
        airports[airportcols].rename(
            columns={col:f'Destination{col}' for col in airportcols}
        ),
    )
    
    routes['geometry'] = routes.apply(
        lambda row: geodesic_line(
            row['SourceLatitude'],row['SourceLongitude'],row['DestinationLatitude'],row['DestinationLongitude'],
            return_distance=True
        ),
        axis=1
    )
    
    routes['Distance'] = routes['geometry'].apply(lambda x: x[1])
    routes['geometry'] = routes['geometry'].apply(lambda x: x[0])
    
    renamedict = {
        'AirlineIATA(2)/ICAO(3)':'AirlineCOD',
        'SourceAirport':'SAirport',
        'SourceAirportID':'SAirportID',
        'DestinationAirport':'DAirport',
        'DestinationAirportID':'DAirportID',
        'SourceLatitude':'SourceLat',
        'SourceLongitude':'SourceLon',
        'SourceCity':'SourceCity',
        'SourceCountry':'SourceCoun',
        'DestinationLatitude':'DestLat',
        'DestinationLongitude':'DestLon',
        'DestinationCity':'DestCity',
        'DestinationCountry':'DestCountr'
    }
    
    routes = gpd.GeoDataFrame(routes,crs=4326).rename(columns=renamedict)
    
    return {
        'airports':airports,
        'airlines':airlines,
        'routes':routes,
        'planes':planes,
        'countries':countries
    }

In [20]:
flights = get_openflights_data()

In [21]:
flights['routes'].to_file('../Resources/OpenFlights/routes')
flights['airports'].to_file('../Resources/OpenFlights/airports')

C:\Users\JCGIRALDO\AppData\Local\Temp\ipykernel_24160\3648686415.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  flights['airports'].to_file('../Resources/OpenFlights/airports')
